# <h3><b>Libraries

In [2]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter
import pymupdf4llm
import glob

from typing import List, Dict, Tuple
from langchain_core.documents import Document
from sentence_transformers import CrossEncoder
import time

c:\Users\saita\anaconda3\envs\portfolio\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# <h3><b>Environment

In [3]:
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
CHROMA_PERSIST_DIRECTORY = "../chroma_db"
COLLECTION_NAME = "machine_learning_modules"

In [3]:
# type(GROQ_API_KEY)

# <h3><b>LLM

In [3]:
llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="openai/gpt-oss-120b",
    temperature=0.2
)
print("✅ LLM initialized!")

✅ LLM initialized!


# <h3><b>Chunking

In [5]:
# # Ekstrak PDF ke Markdown
# md_text = pymupdf4llm.to_markdown(r"D:\Portfolio\rag-llm-education\data\documents\Machine Learning _ Model Klasifikasi pdf.pdf")

# # Simpan hasilnya
# with open("output.md", "w", encoding="utf-8") as f:
#     f.write(md_text)

# Muat dokumen Markdown
# print(f"Jumlah total chunk hierarkis: {len(hierarchical_chunks)}\n")

# for i, chunk in enumerate(hierarchical_chunks):
#     print(f"--- Chunk #{i+1} ---")
#     print(f"Metadata: {chunk.metadata}")
#     # Cetak 100 karakter pertama dari konten untuk pratinjau
#     print(f"Konten: {chunk.page_content[:100]}...\n")

<h5>Chunking bertipe <b>hierarchical

In [25]:
def pdf_chunking_hierarchical(pdf_path: str):
    md_text = pymupdf4llm.to_markdown(pdf_path)

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on
    )
    
    hierarchical_chunks = markdown_splitter.split_text(md_text)
    return hierarchical_chunks

In [4]:
def pdf_chunking_hierarchical(pdf_path: str):
    import pymupdf4llm
    import os
    from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
    from langchain.schema import Document
    
    # Ekstrak nama file dari path
    file_name = os.path.basename(pdf_path)
    # Atau tanpa ekstensi:
    # file_name = os.path.splitext(os.path.basename(pdf_path))[0]
    
    md_text = pymupdf4llm.to_markdown(pdf_path)

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on
    )

    hierarchical_chunks = markdown_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=["\n\n", "\n", ".", "!", "?", " ", ""]
    )

    final_chunks = []
    for chunk in hierarchical_chunks:
        content = chunk.page_content
        sub_chunks = text_splitter.split_text(content)

        for sub in sub_chunks:
            # Gabungkan metadata existing dengan nama file
            metadata = chunk.metadata.copy()
            metadata["source"] = pdf_path  # path lengkap
            metadata["file_name"] = file_name  # nama file saja
            
            final_chunks.append(
                Document(
                    page_content=sub,
                    metadata=metadata
                )
            )

    return final_chunks

# Saat retrieve nanti:
# results = vector_store.similarity_search(query, k=3)
# for doc in results:
#     print(f"Modul: {doc.metadata.get('file_name')}")
#     print(f"Content: {doc.page_content}")

# <h3><b>Embeddings

<ol>
<li><b>paraphrase-multilingual-MiniLM-L12-v2</b>
<p>This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.
<li><b>all-MiniLM-L6-v2</b>
<p>This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
    # model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)
print("✅ Embeddings model loaded")

C:\Users\saita\AppData\Local\Temp\ipykernel_24128\3338366974.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


✅ Embeddings model loaded


# <h3><b>Vector DB

## <h4><b>Store Data to Vector DB Chroma

In [20]:
# pdf_path = "D:/Portfolio/rag-llm-education/data/documents/Machine Learning _ Model Klasifikasi pdf.pdf"
pdf_path = "D:/Codelabs/Roadmap Data Science/E-Book/Cheatsheet Statistika untuk Data Science.pdf"
hierarchical_chunks = pdf_chunking_hierarchical(pdf_path)

In [21]:
CHROMA_PERSIST_DIRECTORY = "../chroma_db"
COLLECTION_NAME = "machine_learning_modules"

def store_to_chromadb(hierarchical_chunks, embeddings, CHROMA_PERSIST_DIRECTORY, COLLECTION_NAME):
    try:
        # Load existing vector store atau buat baru
        try:
            vector_store = Chroma(
                persist_directory=CHROMA_PERSIST_DIRECTORY,
                embedding_function=embeddings,
                collection_name=COLLECTION_NAME
            )
            existing_collection = True
        except:
            vector_store = None
            existing_collection = False
        
        # Cek file_name yang akan ditambahkan
        new_file_name = hierarchical_chunks[0].metadata.get('file_name') if hierarchical_chunks else None
        
        if existing_collection and new_file_name:
            # Cek apakah file sudah ada
            existing_docs = vector_store.get()
            existing_file_names = set()
            
            if existing_docs and existing_docs.get('metadatas'):
                for meta in existing_docs['metadatas']:
                    if meta.get('file_name'):
                        existing_file_names.add(meta['file_name'])
            
            # Jika file sudah ada, hapus data lama
            if new_file_name in existing_file_names:
                print(f"⚠️ File '{new_file_name}' sudah ada. Menghapus data lama...")
                
                # Hapus semua dokumen dengan file_name yang sama
                ids_to_delete = []
                for i, meta in enumerate(existing_docs['metadatas']):
                    if meta.get('file_name') == new_file_name:
                        ids_to_delete.append(existing_docs['ids'][i])
                
                if ids_to_delete:
                    vector_store.delete(ids=ids_to_delete)
                    print(f"✅ Berhasil menghapus {len(ids_to_delete)} dokumen lama dari '{new_file_name}'")
        
        # Tambahkan dokumen baru
        if vector_store is None:
            vector_store = Chroma.from_documents(
                documents=hierarchical_chunks,
                embedding=embeddings,
                persist_directory=CHROMA_PERSIST_DIRECTORY,
                collection_name=COLLECTION_NAME
            )
        else:
            vector_store.add_documents(hierarchical_chunks)
        
        # Hapus duplikat berdasarkan content dan metadata
        print("🔍 Memeriksa duplikat...")
        all_docs = vector_store.get()
        
        if all_docs and all_docs.get('ids'):
            seen = set()
            ids_to_delete = []
            
            for i, (doc_id, content, metadata) in enumerate(zip(
                all_docs['ids'],
                all_docs['documents'],
                all_docs['metadatas']
            )):
                # Buat signature unik dari content + metadata penting
                signature = (
                    content,
                    metadata.get('file_name'),
                    metadata.get('Header 1'),
                    metadata.get('Header 2'),
                    metadata.get('Header 3')
                )
                
                if signature in seen:
                    ids_to_delete.append(doc_id)
                else:
                    seen.add(signature)
            
            # Hapus duplikat
            if ids_to_delete:
                vector_store.delete(ids=ids_to_delete)
                print(f"🗑️ Berhasil menghapus {len(ids_to_delete)} dokumen duplikat")
        
        total_docs = len(vector_store.get()['ids']) if vector_store.get() else len(hierarchical_chunks)
        
        return f"✅ Berhasil menyimpan {len(hierarchical_chunks)} dokumen baru ke ChromaDB.\n📊 Total dokumen di database: {total_docs}"

    except Exception as e:
        return f"❌ Gagal menyimpan ke ChromaDB: {e}"

In [22]:
store_to_chromadb(hierarchical_chunks, embeddings, CHROMA_PERSIST_DIRECTORY, COLLECTION_NAME)

🔍 Memeriksa duplikat...


'✅ Berhasil menyimpan 151 dokumen baru ke ChromaDB.\n📊 Total dokumen di database: 279'

## <h4><b>Load Vector DB

In [7]:
vector_store = Chroma(
    persist_directory=CHROMA_PERSIST_DIRECTORY,
    collection_name=COLLECTION_NAME,
    embedding_function=embeddings
)
print("Database berhasil dimuat.")

C:\Users\saita\AppData\Local\Temp\ipykernel_24128\2346112944.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


Database berhasil dimuat.


## <h4><b>Retriever

In [8]:
retriever = vector_store.as_retriever(
    # search_type="similarity_score_threshold",
    search_type="similarity",
    # search_kwargs={"k": 3, "score_threshold": 0.1}
    search_kwargs={"k": 10}
)

# <h3><b>Reranker V1

In [1]:
# # ========== CELL 8: Load Reranker Model ==========
# print("\n" + "="*70)
# print("🎯 LOADING RERANKER MODEL")
# print("="*70)

# # Load cross-encoder untuk reranking
# reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# print("✅ Reranker model loaded!")
# print("   • Model: cross-encoder/ms-marco-MiniLM-L-6-v2")
# print("   • Type: Cross-encoder (BERT-based)")

In [2]:
# # ========== CELL 1: Enhanced Reranking Function dengan Hierarki ==========
# def rerank_documents_hierarchical(
#     query: str, 
#     documents: List[Document], 
#     top_n: int = 5,
#     use_parent: bool = True,
#     parent_boost: float = 0.1
# ) -> List[Document]:
#     """
#     Rerank documents using cross-encoder dengan support untuk hierarchical chunks
    
#     Args:
#         query: Search query
#         documents: List of retrieved documents (bisa child atau parent chunks)
#         top_n: Number of top documents to return
#         use_parent: Jika True, akan mencoba retrieve parent chunk jika ada
#         parent_boost: Boost score untuk parent chunks (0.1 = +10%)
    
#     Returns:
#         List of reranked documents with scores and hierarchy info
#     """
#     if not documents:
#         return []
    
#     print(f"🔍 Reranking {len(documents)} documents...")
    
#     # Prepare pairs for cross-encoder
#     pairs = []
#     doc_info = []
    
#     for doc in documents:
#         # Check if document has hierarchy metadata
#         chunk_type = doc.metadata.get('chunk_type', 'unknown')
#         has_parent = doc.metadata.get('parent_id') is not None
        
#         pairs.append([query, doc.page_content])
#         doc_info.append({
#             'doc': doc,
#             'chunk_type': chunk_type,
#             'has_parent': has_parent
#         })
    
#     # Get scores from cross-encoder
#     scores = reranker.predict(pairs)
    
#     # Apply hierarchy-aware scoring
#     adjusted_scores = []
#     for score, info in zip(scores, doc_info):
#         adjusted_score = float(score)
        
#         # Boost parent chunks (biasanya lebih informatif)
#         if info['chunk_type'] == 'parent':
#             adjusted_score += parent_boost
        
#         # Store in metadata
#         info['doc'].metadata['relevance_score'] = adjusted_score
#         info['doc'].metadata['raw_score'] = float(score)
        
#         adjusted_scores.append(adjusted_score)
    
#     # Sort by adjusted score (descending)
#     ranked_items = sorted(
#         zip(doc_info, adjusted_scores),
#         key=lambda x: x[1],
#         reverse=True
#     )
    
#     # Return top N
#     top_docs = [item[0]['doc'] for item in ranked_items[:top_n]]
    
#     print(f"✅ Reranking complete! Top score: {ranked_items[0][1]:.4f}")
    
#     return top_docs


# # ========== CELL 2: Function to Expand to Parent Chunks ==========
# def expand_to_parent_chunks(
#     documents: List[Document],
#     vector_store
# ) -> List[Document]:
#     """
#     Expand child chunks ke parent chunks jika tersedia
    
#     Args:
#         documents: List of documents (might be child chunks)
#         vector_store: ChromaDB vector store instance
    
#     Returns:
#         List of documents with parent chunks included
#     """
#     expanded_docs = []
#     parent_ids_seen = set()
    
#     for doc in documents:
#         # Add original document
#         expanded_docs.append(doc)
        
#         # Check if has parent
#         parent_id = doc.metadata.get('parent_id')
        
#         if parent_id and parent_id not in parent_ids_seen:
#             try:
#                 # Retrieve parent chunk by ID
#                 parent_results = vector_store.get(ids=[parent_id])
                
#                 if parent_results and parent_results['documents']:
#                     # Create Document object for parent
#                     parent_doc = Document(
#                         page_content=parent_results['documents'][0],
#                         metadata=parent_results['metadatas'][0] if parent_results['metadatas'] else {}
#                     )
#                     expanded_docs.append(parent_doc)
#                     parent_ids_seen.add(parent_id)
                    
#             except Exception as e:
#                 print(f"⚠️ Could not retrieve parent {parent_id}: {e}")
    
#     return expanded_docs


# # ========== CELL 3: Advanced Retrieval with Hierarchical Reranking ==========
# def retrieve_and_rerank_hierarchical(
#     query: str,
#     vector_store,
#     k_initial: int = 10,
#     top_n_final: int = 5,
#     expand_parents: bool = True,
#     use_parent_boost: bool = True
# ) -> Tuple[List[Document], Dict]:
#     """
#     Complete retrieval pipeline dengan hierarchical support dan reranking
    
#     Returns:
#         Tuple of (reranked_documents, metrics_dict)
#     """
#     metrics = {}
    
#     print("\n" + "="*70)
#     print("🔍 HIERARCHICAL RETRIEVAL + RERANKING")
#     print("="*70)
    
#     # Step 1: Initial retrieval
#     print(f"\n📥 Step 1: Initial retrieval (k={k_initial})")
#     start = time.time()
#     initial_results = vector_store.similarity_search(query, k=k_initial)
#     retrieval_time = time.time() - start
#     metrics['retrieval_time'] = retrieval_time
#     metrics['initial_count'] = len(initial_results)
    
#     print(f"   ✅ Retrieved {len(initial_results)} chunks")
#     print(f"   ⏱️  Time: {retrieval_time*1000:.2f}ms")
    
#     # Analyze chunk types
#     chunk_types = {}
#     for doc in initial_results:
#         chunk_type = doc.metadata.get('chunk_type', 'unknown')
#         chunk_types[chunk_type] = chunk_types.get(chunk_type, 0) + 1
    
#     print(f"   📊 Chunk types: {chunk_types}")
    
#     # Step 2: Expand to parents (optional)
#     if expand_parents:
#         print(f"\n🔼 Step 2: Expanding to parent chunks")
#         start = time.time()
#         expanded_results = expand_to_parent_chunks(initial_results, vector_store)
#         expand_time = time.time() - start
#         metrics['expand_time'] = expand_time
#         metrics['expanded_count'] = len(expanded_results)
        
#         print(f"   ✅ Expanded to {len(expanded_results)} chunks (includes parents)")
#         print(f"   ⏱️  Time: {expand_time*1000:.2f}ms")
        
#         documents_to_rerank = expanded_results
#     else:
#         documents_to_rerank = initial_results
#         metrics['expand_time'] = 0
#         metrics['expanded_count'] = len(initial_results)
    
#     # Step 3: Rerank
#     print(f"\n🎯 Step 3: Reranking to top {top_n_final}")
#     start = time.time()
#     reranked_results = rerank_documents_hierarchical(
#         query=query,
#         documents=documents_to_rerank,
#         top_n=top_n_final,
#         use_parent=True,
#         parent_boost=0.1 if use_parent_boost else 0
#     )
#     rerank_time = time.time() - start
#     metrics['rerank_time'] = rerank_time
#     metrics['final_count'] = len(reranked_results)
    
#     print(f"   ⏱️  Time: {rerank_time*1000:.2f}ms")
    
#     # Total time
#     total_time = retrieval_time + metrics['expand_time'] + rerank_time
#     metrics['total_time'] = total_time
    
#     print(f"\n⏱️  TOTAL TIME: {total_time*1000:.2f}ms")
#     print("="*70)
    
#     return reranked_results, metrics


# # ========== CELL 4: Display Results Function ==========
# def display_reranked_results(documents: List[Document], query: str = None):
#     """
#     Display reranked results dengan hierarchy information
#     """
#     if query:
#         print(f"\n❓ Query: '{query}'")
    
#     print(f"\n📚 Top {len(documents)} Results:\n")
#     print("="*70)
    
#     for i, doc in enumerate(documents, 1):
#         # Extract metadata
#         page = doc.metadata.get('page', 'N/A')
#         chunk_type = doc.metadata.get('chunk_type', 'unknown')
#         chunk_id = doc.metadata.get('chunk_id', 'N/A')
#         parent_id = doc.metadata.get('parent_id', None)
#         relevance_score = doc.metadata.get('relevance_score', 0)
#         raw_score = doc.metadata.get('raw_score', 0)
        
#         # Header
#         print(f"\n{i}. 📄 Page {page if page == 'N/A' else page+1} | Type: {chunk_type.upper()}")
#         print(f"   ID: {chunk_id}")
#         if parent_id:
#             print(f"   Parent ID: {parent_id}")
        
#         # Scores
#         print(f"   🎯 Relevance Score: {relevance_score:.4f} (raw: {raw_score:.4f})")
        
#         # Content preview
#         content_preview = doc.page_content[:200] if len(doc.page_content) > 200 else doc.page_content
#         print(f"   📝 Content:\n   {content_preview}...")
        
#         # Hierarchy indicator
#         if chunk_type == 'parent':
#             print(f"   🔼 [PARENT CHUNK - Contains broader context]")
#         elif chunk_type == 'child':
#             print(f"   🔽 [CHILD CHUNK - Focused content]")
        
#         print("-"*70)

In [3]:
# # ========== CELL 5: Test Hierarchical Retrieval + Reranking ==========
# print("\n" + "="*70)
# print("🧪 TEST: HIERARCHICAL RETRIEVAL + RERANKING")
# print("="*70)

# # Test query
# test_query = "Apa itu overfitting?"

# # Retrieve and rerank
# reranked_docs, metrics = retrieve_and_rerank_hierarchical(
#     query=test_query,
#     vector_store=vector_store,
#     k_initial=10,           # Retrieve 10 chunks initially
#     top_n_final=5,          # Return top 5 after reranking
#     expand_parents=True,    # Include parent chunks
#     use_parent_boost=True   # Boost parent chunk scores
# )

# # Display results
# display_reranked_results(reranked_docs, query=test_query)

# # Display metrics
# print("\n📊 METRICS:")
# print("="*70)
# for key, value in metrics.items():
#     if 'time' in key:
#         print(f"   • {key}: {value*1000:.2f}ms")
#     else:
#         print(f"   • {key}: {value}")
# print("="*70)

# <h3><b>Reranker V2

In [4]:
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain_community.cross_encoders import HuggingFaceCrossEncoder

In [5]:
# # --- 4. Inisialisasi Model Reranker (LANGKAH BARU) ---
# print("Memuat model reranker...")
# # Gunakan model cross-encoder dari BAAI yang bagus untuk multilingual
# reranker_model = HuggingFaceCrossEncoder(
#     model_name="BAAI/bge-reranker-base",
#     model_kwargs={'device': 'cpu'} # Ganti ke 'cuda' jika Anda memiliki GPU
# )
# print("✅ Model reranker berhasil dimuat.")

In [6]:
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=reranker_model, 
#     base_retriever=retriever
# )
# print("✅ Compression retriever dengan reranker telah dibuat.")

In [7]:
# # --- 6. Contoh Penggunaan ---
# # Sekarang, gunakan `compression_retriever` untuk melakukan pencarian
# query = "Apa itu model klasifikasi dalam machine learning?"
# print(f"\nMelakukan pencarian dengan query: '{query}'")

# # Gunakan .invoke() untuk mendapatkan dokumen yang sudah di-rerank
# reranked_docs = compression_retriever.invoke(query)

# print(f"\nDitemukan {len(reranked_docs)} dokumen yang sudah di-rerank:\n")

# # Tampilkan hasilnya
# for i, doc in enumerate(reranked_docs):
#     # Perhatikan bahwa metadata dari chunking hierarkis Anda masih ada
#     print(f"--- Dokumen #{i+1} ---")
#     print(f"Metadata: {doc.metadata}")
#     # Tampilkan sebagian konten
#     print(f"Konten: {doc.page_content[:200]}...")
#     print("-" * 20)

# <h4><b>Delete Collection

In [125]:
import chromadb

try:
    # Inisialisasi klien Chroma yang terhubung ke folder persistensi
    client = chromadb.PersistentClient(path=CHROMA_PERSIST_DIRECTORY)

    # Hapus koleksi berdasarkan nama
    client.delete_collection(name=COLLECTION_NAME)
    
    print(f"✅ Koleksi '{COLLECTION_NAME}' berhasil dihapus dari database.")

except Exception as e:
    print(f"❌ Gagal menghapus koleksi: {e}")
    print("Mungkin koleksi tidak ditemukan atau database belum ada.")

✅ Koleksi 'machine_learning_modules' berhasil dihapus dari database.


# <h3><b>Prompt

In [17]:
template = """Kamu adalah asisten AI yang membantu menjawab pertanyaan berdasarkan konteks yang diberikan.

Konteks:
{context}

Pertanyaan: {question}

Instruksi:
- Jawab berdasarkan konteks yang diberikan
- Jika tidak ada informasi di konteks, katakan "Maaf, saya tidak menemukan informasi tersebut dalam dokumen"
- Berikan jawaban yang jelas dan ringkas
- Gunakan bahasa Indonesia yang baik

Jawaban:"""

prompt = ChatPromptTemplate.from_template(template)
print("✅ Prompt template created")

✅ Prompt template created


In [8]:
system_prompt = """instructions:
task: Tugasmu adalah menjawab pertanyaan dari mahasiswa berdasarkan dokumen modul ajar yang diberikan. Gunakan informasi dari dokumen untuk memberikan jawaban yang akurat dan relevan.
persona: Kamu adalah seorang dosen yang menjawab pertanyaan mahasiswa dengan detail dan jelas.
method: Untuk menjawab pertanyaan, ikuti langkah-langkah berikut:
1. Baca pertanyaan mahasiswa dengan seksama.
2. Cari informasi yang relevan dari dokumen modul ajar yang diberikan.
3. Susun jawaban yang komprehensif dan mudah dipahami berdasarkan informasi tersebut.
4. Jika informasi tidak cukup, katakan bahwa kamu tidak memiliki cukup data untuk menjawab pertanyaan tersebut.
output-length: Jawaban harus padat sesuai dengan yang ada di dokumen.
output-format: sebuah paragraf.
inclusion: Penjelasan dari dokumen modul ajar yang relevan dengan pertanyaan.
handle-unknown: Jika informasi yang diberikan tidak cukup untuk menjawab pertanyaan, katakan 'Maaf, saya tidak memiliki cukup informasi untuk menjawab pertanyaan ini.'
"""

user_prompt = """context:
  relevant documents: "{docs}"
  question: "{query}"
"""

# --- Buat prompt LangChain ---
# prompt = ChatPromptTemplate.from_template(prompt_template)
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])
print("✅ Prompt template loaded from JSON successfully")


✅ Prompt template loaded from JSON successfully


In [25]:
vector_store.persist()

In [9]:
# Ambil semua metadata
all_data = vector_store.get()

# Ambil daftar metadata dari semua dokumen
metadatas = all_data["metadatas"]

# Ambil hanya field 'file_name' dan buang duplikat
file_names = list({meta.get("file_name") for meta in metadatas if meta.get("file_name")})

# Tampilkan hasil
print("📁 Daftar file yang tersimpan:")
for i, name in enumerate(file_names, 1):
    print(f"{i}. {name}")

📁 Daftar file yang tersimpan:
1. 500 Pertanyaan dan Jawaban Interview Data Scientist.pdf
2. Machine Learning _ Model Klasifikasi pdf.pdf
3. Cheatsheet Visualisasi Data dengan Seaborn_ Teori, Penerapan, dan Code .pdf
4. Cheatsheet Statistika untuk Data Science.pdf


# <h3><b>RAG Chain

In [9]:
def format_docs(docs):
    """Format dokumen untuk context"""
    return "\n\n".join(doc.page_content for doc in docs)

# Chain menggunakan LCEL (LangChain Expression Language)
rag_chain = (
    {"docs": retriever | format_docs, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG Chain created!")

✅ RAG Chain created!


# <h3><b>Query Function

In [10]:
def ask_question(question):
    """Fungsi untuk bertanya ke chatbot"""
    print(f"Pertanyaan: {question}")
    
    # Get answer
    answer = rag_chain.invoke(question)
    
    # Get source documents
    docs = retriever.invoke(question)
    print("📚 Sumber Dokumen:")
    for i, doc in enumerate(docs, 1):
        print(f"Metadata: {doc.metadata}")
        print(f"Data:\n{i}. {doc.page_content}...")
    return answer


# <h3><b>Test Retrieve

In [134]:
# --- 3. LAKUKAN PENCARIAN ---
user_query = "Apa itu overfitting?"
expanded_query = "definisi overfitting dalam machine learning, penyebab overfitting, dampak overfitting pada akurasi model, teknik deteksi overfitting (mis‑fit training vs. validation), metode pencegahan dan mitigasi overfitting (regularisasi, dropout, early stopping, cross‑validation, pruning) serta contoh kasus overfitting pada model supervised learning."
print(f"\nMelakukan pencarian untuk query: '{expanded_query}'")

relevant_docs = retriever.invoke(expanded_query)


Melakukan pencarian untuk query: 'definisi overfitting dalam machine learning, penyebab overfitting, dampak overfitting pada akurasi model, teknik deteksi overfitting (mis‑fit training vs. validation), metode pencegahan dan mitigasi overfitting (regularisasi, dropout, early stopping, cross‑validation, pruning) serta contoh kasus overfitting pada model supervised learning.'


In [138]:
# --- 4. TAMPILKAN HASIL ---
if relevant_docs:
    print(f"\nDitemukan {len(relevant_docs)} dokumen relevan:")
    for i, doc in enumerate(relevant_docs):
        print(f"\n--- Dokumen #{i+1} ---")
        print(f"Metadata: {doc.metadata}")
        print(f"Filename: {doc.metadata['file_name']}...")
        print(f"Konten: {doc.page_content}...")
else:
    print("Tidak ada dokumen relevan ditemukan.")


Ditemukan 10 dokumen relevan:

--- Dokumen #1 ---
Metadata: {'source': 'D:/Portfolio/rag-llm-education/data/documents/Machine Learning _ Model Klasifikasi pdf.pdf', 'Header 1': '**Bab 1: Pendahuluan: Mengenal Dunia** **Klasifikasi dalam Machine Learning**', 'file_name': 'Machine Learning _ Model Klasifikasi pdf.pdf', 'Header 3': '**1.4. Terminologi Penting dalam Klasifikasi**'}
Filename: Machine Learning _ Model Klasifikasi pdf.pdf...
Konten: mengidentifikasi pola dan hubungan antara fitur dan label.  
**Data Uji (Testing Data)**  
Data uji adalah kumpulan data yang digunakan untuk menguji kinerja model ML setelah
proses pelatihan. Data uji terdiri dari fitur tetapi tidak menyertakan label. Model ML akan
membuat prediksi label untuk data uji, yang kemudian dapat dibandingkan dengan label
sebenarnya (jika diketahui) untuk menghitung metrik evaluasi.  
7  
**Overfitting**  
Overfitting terjadi ketika model ML terlalu baik dalam mempelajari pola dan detail dari data
latih, sehingga menja

In [80]:
first_pass_docs = retriever.get_relevant_documents("Bagaimana mengatasi overfitting dalam machine learning?")

selector_prompt = """
Dari dokumen berikut, pilih yang paling menjawab pertanyaan pengguna: "{query}".
Berikan hanya teks yang paling relevan.
docs: "{docs}"
"""
selected_docs = llm.invoke(selector_prompt.format(query="Bagaimana mengatasi overfitting dalam machine learning?", docs=first_pass_docs))


In [82]:
first_pass_docs

[Document(metadata={'Header 1': '**Bab 1: Pendahuluan: Mengenal Dunia** **Klasifikasi dalam Machine Learning**', 'Header 3': '**1.4. Terminologi Penting dalam Klasifikasi**'}, page_content='mengidentifikasi pola dan hubungan antara fitur dan label.  \n**Data Uji (Testing Data)**  \nData uji adalah kumpulan data yang digunakan untuk menguji kinerja model ML setelah\nproses pelatihan. Data uji terdiri dari fitur tetapi tidak menyertakan label. Model ML akan\nmembuat prediksi label untuk data uji, yang kemudian dapat dibandingkan dengan label\nsebenarnya (jika diketahui) untuk menghitung metrik evaluasi.  \n7  \n**Overfitting**  \nOverfitting terjadi ketika model ML terlalu baik dalam mempelajari pola dan detail dari data\nlatih, sehingga menjadi kurang mampu untuk membuat prediksi yang akurat pada data\nbaru yang belum pernah dilihat sebelumnya. Overfitting biasanya terjadi ketika model ML\nterlalu kompleks dan mencoba untuk menyesuaikan diri dengan setiap titik data dalam data  \nlatih.

In [81]:
selected_docs.content

'**Regularisasi**  \nRegularisasi adalah teknik yang digunakan untuk mengurangi overfitting dalam model ML dengan menambahkan hukuman (penalty) terhadap kompleksitas model. Regularisasi mendorong model untuk menjadi lebih sederhana dan menghindari penyesuaian yang berlebihan terhadap data latih.  \n\n**Cross‑Validation**  \nCross‑validation adalah teknik yang digunakan untuk mengestimasi kinerja model ML pada data baru yang belum pernah dilihat sebelumnya. Dalam cross‑validation, data latih dibagi menjadi beberapa lipatan (folds), di mana model ML dilatih pada sebagian lipatan dan diuji pada lipatan yang tersisa. Proses ini diulangi beberapa kali dengan lipatan yang berbeda.'

In [49]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

prompt_query = PromptTemplate.from_template("""
Kamu adalah asisten pencarian. Reformulasikan query berikut agar lebih cocok untuk pencarian dokumen teknis machine learning.

Query asli: "{query}"
Kata Kunci yang  Dicari:

Berikan versi yang lebih spesifik dan kaya kata kunci, tanpa mengubah makna.
""")

query_rewriter = LLMChain(llm=llm, prompt=prompt_query)

user_query = "apa itu overfitting"
expanded_query = query_rewriter.run(query=user_query)

print(expanded_query)

**Query yang direformulasi:**

> “definisi overfitting dalam machine learning, penyebab overfitting, dampak overfitting pada akurasi model, teknik deteksi overfitting (mis‑fit training vs. validation), metode pencegahan dan mitigasi overfitting (regularisasi, dropout, early stopping, cross‑validation, pruning) serta contoh kasus overfitting pada model supervised learning.”


# <h3><b>Test Chatbot

In [11]:
jawaban = ask_question("Apa itu decision tree dan bagaimana cara kerjanya?")
print(f"Jawaban: {jawaban}")

Pertanyaan: Apa itu decision tree dan bagaimana cara kerjanya?
📚 Sumber Dokumen:
Metadata: {'Header 1': '**Bab 5: Decision Trees: Klasifikasi** **Berbasis Struktur Pohon Keputusan**', 'source': 'D:/Portfolio/rag-llm-education/data/documents/Machine Learning _ Model Klasifikasi pdf.pdf', 'Header 3': '**5.4 Kasus yang Direkomendasikan untuk Menggunakan** **Decision Trees**', 'file_name': 'Machine Learning _ Model Klasifikasi pdf.pdf'}
Data:
1. Decision Trees sangat cocok untuk:  
1. Klasifikasi data yang memiliki fitur kategorikal atau numerik
2. Masalah yang memerlukan interpretasi yang mudah dan bisa dijelaskan
3. Ketika overfitting bisa dikendalikan dengan membatasi kedalaman pohon atau
prunning...
Metadata: {'Header 3': '**5.1 Pendahuluan**', 'file_name': 'Machine Learning _ Model Klasifikasi pdf.pdf', 'source': 'D:/Portfolio/rag-llm-education/data/documents/Machine Learning _ Model Klasifikasi pdf.pdf', 'Header 1': '**Bab 5: Decision Trees: Klasifikasi** **Berbasis Struktur Pohon Ke

In [ ]:
ask_question("Siapa presiden Indonesia?")


❓ Pertanyaan: Siapa presiden Indonesia?

💬 Jawaban:
Maaf, saya tidak menemukan informasi tersebut dalam dokumen.

📚 Sumber Dokumen:

1. Akurasi adalah metrik yang paling sederhana dan intuitif. Akurasi menggambarkan
seberapa sering model membuat prediksi yang benar.  
Akurasi = (Jumlah...

2. Contoh: Mengklasifikasikan jenis kelamin berdasarkan tinggi, berat, dan ukuran sepatu.  
24...

3. JST cocok untuk digunakan pada kasus-kasus berikut:  
1. Klasifikasi gambar dan pengenalan pola
2. Analisis teks dan pemrosesan bahasa alami  
36  
3....



'Maaf, saya tidak menemukan informasi tersebut dalam dokumen.'